__Aggregator__

The aggregator allows us to load a set of results which were generated by running the same pipeline and phase.

In the 'example_runner.py', we fitted two images with the pipeline, 'pl__lens_sersic_sie_shear_source_x1_sersic' in the file 'pipelines/simple/lens_sersic_sie_shear_source_sersic.py'. The results of this fit are distributed in the autolens_workspace and can be found in the 'autolens_workspace/output/example' folder.

We can load all the results from any phase of this pipeline using the aggregator, enabling us to manipulate those results in Python to plot figures, interpret results, check specific values, etc.

In [1]:
import autofit as af


To begin, we setup the path to the workspace and our output folder.

In [2]:
workspace_path = "/home/jammy/PycharmProjects/PyAutoLens/workspace/"
output_path = workspace_path + "output/"

Now we'll ue this path to explicitly set the config path and output path.

In [3]:
af.conf.instance = af.conf.Config(
    config_path=workspace_path + "config", output_path=output_path
)

To use the aggregator we have to supply it with the pipeline name we want to load results from and the phase name of that pipeline we want to load results from. Lets load the results of the pipeline's final phase.

In [5]:
pipeline = "pl__lens_sersic_sie_shear_source_x1_sersic"
phase = "phase_3__lens_sersic_sie_shear_source_sersic"

First, we create an instance of the aggregator, which takes the output path as input, telling it where to load results from.

In [6]:
aggregator = af.Aggregator(directory=output_path)

We can load the results of all phases. The results are just a Python string that summarizes every fit's lens model.

Other than printing this string for a quick inspection of all results, this doesn't offer too much utility, but it can be a handy way to quickly inspect all results.

In [7]:
results = aggregator.model_results(pipeline=pipeline, phase=phase)

print(results)

We can create is a list of the optimizer classes of every phase. When you make a phase in a pipeline, you are probably used to seeing the following line:

optimizer_class=af.MultiNest

This line is telling the Phase which non-linear optimizer to use, in this case MultiNest. Every optimizer is created as an instance of a class in Python, which has methods built-in allowing one to manipulate its results.

In [8]:
optimizers = aggregator.optimizers_with(pipeline=pipeline, phase=phase)

First, if we print the optimizers we'll see that we have a Python list of two optimizers. These are the optimizers of phase 3 of our pipeline for the two images we modeled.

If we had modeled 100 images, we'd have 100 optimizers. Clearly, with that in mind, for the code below you should be thinking how important the aggregator in manipulating large lensing data-sets.

In [9]:
print(optimizers)

[]


Whats particularly powerful is that every optimizer now acts as an interface to the MultiNest chains on the hard disk. This means we can use the optimizers to, for example, create an instance of the most likely (e.g. highest likelihood) model of every fit to an image.

In [ ]:
most_likely_model_parameters = list(
    map(lambda opt: opt.most_likely_model_parameters, optimizers)
)

print(most_likely_model_parameters)

This creates a 1D Python list of all the model parameters. However, this isn't that much use. Afterall, which values correspond to which parameters? To find out, you'd have to directly compare to the files output on the hard-disk, which isn't ideal. However, we can instead create a model instance of every fit.

In [ ]:
most_likely_model_instances = list(
    map(lambda opt: opt.most_likely_model_instance, optimizers)
)

print(most_likely_model_instances)

This has created a list of 2 ModelInstance classes. A model instance uses the naming convention of the GalaxyModel in the pipeline to store the results. For example, in this phase:

- The lens galaxy was called 'lens', its light profile 'light', mass profile 'mass' and shear 'shear'.
- The source galaxy was called 'source' and its light profile 'light'.

We can access all of these components of the model in our model instance.

In [ ]:
print(most_likely_model_instances[0].galaxies.lens.light)
print(most_likely_model_instances[0].galaxies.lens.light.sersic_index)
print(most_likely_model_instances[1].galaxies.lens.light.sersic_index)
# print(most_likely_model_instances[0].galaxies.source.light)

For a large number of images, this can give us a really easy way to, for example, retrieve a list of the Sersic index of every most likely lens model.

In [ ]:
sersic_indexes = list(
    map(lambda ml: ml.galaxies.lens.light.sersic_index, most_likely_model_instances)
)

print(sersic_indexes)

We can also access the 'most probable' model, which is the model computed by marginalizing over the MultiNest samples of every parameter in 1D and taking the median of this PDF.

In [ ]:
most_probable_model_parameters = list(
    map(lambda opt: opt.most_probable_model_parameters, optimizers)
)
print(most_probable_model_parameters)

This again produces a 1D Python list where it is difficult to know which parameters are which. Fortunately, we can create an equivalent ModelInstance of the most probable model.

In [ ]:
most_probable_model_instances = list(
    map(lambda opt: opt.most_probable_model_instance, optimizers)
)

print(most_probable_model_instances)

Again, we could manipulate these to get all of the most probable values of a given parameter.

In [ ]:
einstein_radii = list(
    map(lambda mp: mp.galaxies.lens.mass.einstein_radius, most_probable_model_instances)
)

print(einstein_radii)

We can also access the inferred errors of every parameter, by inputting a sigma significance. The errors represent the values at the % of the PDF from the median (e.g. 3 sigma errors are at 0.3% and 99.7%)

In [ ]:
upper_error_instances = list(
    map(
        lambda opt: opt.model_errors_instance_at_upper_sigma_limit(sigma_limit=3.0),
        optimizers,
    )
)
lower_error_instances = list(
    map(
        lambda opt: opt.model_errors_instance_at_lower_sigma_limit(sigma_limit=3.0),
        optimizers,
    )
)

print(upper_error_instances)
print(lower_error_instances)

Again, we can manipulate these to get the errors on our Einstein radii

In [ ]:
einstein_radii_upper_errors = list(
    map(lambda u: u.galaxies.lens.mass.einstein_radius, upper_error_instances)
)
einstein_radii_lower_errors = list(
    map(lambda l: l.galaxies.lens.mass.einstein_radius, lower_error_instances)
)

print(einstein_radii_upper_errors)
print(einstein_radii_lower_errors)

It is worth noting that the results returned by the ModelInstance are instances of PyAutoLens classes.

For example, the lens 'light' component' is an instance of an EllipticalSersic, meaning we can use the methods of the EllipticalSersic class.

In [ ]:
luminosities = list(
    map(
        lambda mp: mp.galaxies.lens.light.luminosity_within_circle_in_units(radius=1.0),
        most_probable_model_instances,
    )
)
print(luminosities)

And, of course, the 'lens' is an instance of the Galaxy class.

In [ ]:
masses = list(
    map(
        lambda mp: mp.galaxies.lens.mass_within_circle_in_units(radius=1.0),
        most_probable_model_instances,
    )
)
print(masses)